In [1]:
import pandas as pd
import json
import csv

/tmp/ipykernel_31220/2800771450.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
with open('/home/ashkan/Code/testimonial_llm_analysis/indus_clean.json') as f:
    indust = json.load(f)

In [ ]:
def convert_to_csv(indust, filename):
    # Define the fieldnames for the CSV file
    fieldnames = ['story_id', 'company_name', 'industry_1', 'industry_2', 'industry_3']

    # Open the CSV file in write mode
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header row
        writer.writeheader()

        # Write each row of data
        for item in indust:
            row = {
                'story_id': item['story_id'],
                'company_name': item.get('company_name', ''),
                'industry_1': item['industry'][0] if len(item['industry']) >= 1 else '',
                'industry_2': item['industry'][1] if len(item['industry']) >= 2 else '',
                'industry_3': item['industry'][2] if len(item['industry']) >= 3 else ''
            }
            writer.writerow(row)

# Call the function to convert the 'indust' list to a CSV file
convert_to_csv(indust, 'indust.csv')

In [ ]:
# Read the CSV file into a dataframe
df = pd.read_csv('/home/ashkan/Code/testimonial_llm_analysis/indust.csv')

# Lowercase every column
df = df.apply(lambda x: x.str.lower())

In [ ]:
df['industry_1'] = df['industry_1'].apply(lambda x: 'other' if x == 'o' else x)
df.loc[df['industry_1'] == 'other', ['industry_2', 'industry_3']] = ""

In [ ]:
df_17 = pd.read_csv('/mnt/c/Users/alotf/OneDrive - University of Surrey/[1] Raw Data/slr_paper/companies-2023.csv')

In [ ]:
# Open the combined_data.json file
with open('/home/ashkan/Code/customer_stories/results/combined_data.json') as f:
    data = json.load(f)

# Extract the id and company keys' values
id_company_data = [(item['id'], item['company']) for item in data]

# Define the fieldnames for the CSV file
fieldnames = ['id', 'company']

# Save the data as a CSV file
with open('id_company.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(fieldnames)
    writer.writerows(id_company_data)

In [ ]:
df_org = pd.read_csv(csvfile.name)
df_org = df_org.apply(lambda x: x.str.lower())

In [ ]:
df_merge = df.merge(df_org[['id', 'company']], left_on='story_id', right_on='id', how='left').drop(columns='id')
df_merge.rename(columns={'company': 'company_org'}, inplace=True)
df_merge = df_merge[['story_id', 'company_name', 'company_org', 'industry_1', 'industry_2', 'industry_3']]

In [ ]:
df_subset = df_merge[df_merge['company_name'].isna()]
df_subset

In [ ]:
df_merge.to_csv('df_merge.csv', index=False)

# Delete empty stories

In [2]:
# Open the combined_data.json file
with open('/home/ashkan/Code/customer_stories/results/combined_data.json') as f:
    data = json.load(f)

# Get the number of entries before deletion
num_entries_before = len(data)

# Check if the 'content' key is empty and delete the entry if it is
data = [entry for entry in data if entry.get('content')]

# Get the number of entries after deletion
num_entries_after = len(data)

Number of entries before deletion: 2865
Number of entries after deletion: 2723


In [4]:
# Print the number of entries before and after deletion
print(f"Number of entries before deletion: {num_entries_before}")
print(f"Number of entries after deletion: {num_entries_after}")

Number of entries before deletion: 2865
Number of entries after deletion: 2723


In [6]:
# Specify the file path for the new JSON file
new_file_path = '/home/ashkan/Code/customer_stories/results/data_wo_empty.json'

# Save the data list as a JSON file
with open(new_file_path, 'w') as f:
    json.dump(data, f)

In [7]:
text = "   For Mayborn Group, the company behind the Tommee Tippee brand of baby care products, brand loyalty and market presence are important priorities. Tracking critical indicators in these areas and reaching a single source of truth is essential, and it\u2019s an area where high-quality data is key. Acquiring this data quickly and accurately from such a broad range of sources and locations is, however, a major task. With Qlik, Mayborn can now access reliable data in consistently useable formats to create high value analytics. Using Qlik Data Integration, Mayborn can access data from multiple locations to feed into its data warehouse. And Qlik Cloud\u00ae Analytics generates dashboards that tap into every part of Mayborn\u2019s operation to deliver key insights. \n\n \n"

In [8]:
print(len(text))

758


In [9]:
# Filter the data list to remove entries with content length less than 1000 characters
data = [entry for entry in data if len(entry.get('content', '')) >= 1000]

# Update the number of entries after deletion
num_entries_after = len(data)

# Print the number of entries before and after deletion
print(f"Number of entries before deletion: {num_entries_before}")
print(f"Number of entries after deletion: {num_entries_after}")

Number of entries before deletion: 2865
Number of entries after deletion: 2583


In [10]:
# Save the updated data list as a JSON file
with open('/home/ashkan/Code/customer_stories/results/data_final.json', 'w') as f:
    json.dump(data, f)